In [1]:
import numpy as np
import pandas as pd
import duckdb


from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# Pytorch and Pytorch Geometric
import torch
from torch_geometric.data import Data
from torch.nn import Linear, ReLU
from torch_geometric.nn import Sequential, GCNConv
from torch_geometric.nn import global_mean_pool, global_add_pool
import torch.nn.functional as Fun


import warnings
warnings.filterwarnings("ignore")


In [2]:
train_path = '/home/gyn/Kaggle/train.parquet'

con = duckdb.connect()

df_mini = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 100
                       ) UNION ALL
                       (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 100
                       )
                        """).df()

con.close()

In [3]:
def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """
    if x not in permitted_list:
        x = permitted_list[-1]
    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]
    return binary_encoding

In [4]:
"""
这部分内容对原子的特征进行了提取：将分子中的每个原子转换为特征向量（原子的种类、键合状态、手性情况）
"""
def get_atom_features(atom, 
                      use_chirality = True, 
                      hydrogens_implicit = True):
    """
    Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.
    """

    # define list of permitted atoms
    
    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']
    
    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms
    
    # compute atom features
    
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)
    
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    
    is_in_a_ring_enc = [int(atom.IsInRing())]
    
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    
    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]
    
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]
    
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled
                                    
    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc
    
    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

In [5]:
def get_bond_features(bond, 
                      use_stereochemistry = True):
    """
    Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
    """

    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]

    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)
    
    bond_is_conj_enc = [int(bond.GetIsConjugated())]
    
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc
    
    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

In [6]:
def create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, y):
    """
    Inputs:
    
    x_smiles = [smiles_1, smiles_2, ....] ... a list of SMILES strings
    y = [y_1, y_2, ...] ... a list of numerial labels for the SMILES strings (such as associated pKi values)
    
    Outputs:
    
    data_list = [G_1, G_2, ...] ... a list of torch_geometric.data.Data objects which represent labeled molecular graphs that can readily be used for machine learning
    
    """
    
    data_list = []
    
    for (smiles, y_val) in zip(x_smiles, y):
        
        # convert SMILES to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)

        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

        # construct node feature matrix X of shape (n_nodes, n_node_features)
        X = np.zeros((n_nodes, n_node_features))

        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)
            
        X = torch.tensor(X, dtype = torch.float)
        
        # construct edge index array E of shape (2, n_edges)
        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
        torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim = 0)
        
        # construct edge feature array EF of shape (n_edges, n_edge_features)
        EF = np.zeros((n_edges, n_edge_features))
        
        for (k, (i,j)) in enumerate(zip(rows, cols)):
            
            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))
        
        EF = torch.tensor(EF, dtype = torch.float)
        
        # construct label tensor
        y_tensor = torch.tensor(np.array([y_val]), dtype = torch.float)
        
        # construct Pytorch Geometric data object and append to data list
        data_list.append(Data(x = X, edge_index = E, edge_attr = EF, y = y_tensor))

    return data_list

In [11]:
## model
class  GCN(torch.nn.Module):
    """Graph Convolutional Network class with 3 convolutional layers and a linear layer"""

    def __init__(self, dim_h):
        """init method for GCN

        Args:
            dim_h (int): the dimension of hidden layers
        """
        super().__init__()
        self.conv1 = GCNConv(79, dim_h)
        self.conv2 = GCNConv(dim_h, dim_h)
        self.conv3 = GCNConv(dim_h, dim_h)
        self.lin = torch.nn.Linear(dim_h, 1)

    def forward(self, data):
        e = data.edge_index
        x = data.x
        print(e)
        print(x)
        print(data.batch)
        
        x = self.conv1(x, e)
        x = x.relu()
        x = self.conv2(x, e)
        x = x.relu()
        x = self.conv3(x, e)
        x = global_mean_pool(x, data.batch)

        x = Fun.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return torch.sigmoid(x)
    


In [12]:
# canonical training loop for a Pytorch Geometric GNN model gnn_model
import torch.nn as nn
# create list of molecular graph objects from list of SMILES x_smiles and list of labels y
data_list = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(df_mini['molecule_smiles'].tolist(), df_mini['binds'].tolist())

gnn_model = GCN(dim_h =64)



In [13]:
print(data_list[0])

Data(x=[47, 79], edge_index=[2, 102], edge_attr=[102, 10], y=[1])


In [ ]:
from torch_geometric.loader import DataLoader


# create dataloader for training
dataloader = DataLoader(dataset = data_list, batch_size = 4, shuffle =True)

# define loss function
loss_function = nn.MSELoss()

# define optimiser
optimiser = torch.optim.Adam(gnn_model.parameters(), lr = 1e-3)

# loop over 10 training epochs
for epoch in range(10):

    # set model to training mode
    gnn_model.train()

    # loop over minibatches for training
    for batch in dataloader:
        print(batch.head(20))

        # compute current value of loss function via forward pass
        output = gnn_model(batch)
        loss_function_value = loss_function(output[:,0], torch.tensor(batch.y, dtype = torch.float32))

        # set past gradient to zero
        optimiser.zero_grad()

        # compute current gradient via backward pass
        loss_function_value.backward()

        # update model weights using gradient and optimisation method
        optimiser.step()

DataBatch(x=[158, 79], edge_index=[2, 344], edge_attr=[344, 10], y=[4], batch=[158], ptr=[5])
tensor([[  0,   1,   1,   1,   2,   2,   3,   3,   3,   4,   4,   5,   5,   5,
           6,   6,   7,   7,   7,   8,   8,   9,   9,  10,  10,  10,  11,  11,
          12,  12,  13,  13,  14,  14,  15,  15,  15,  16,  16,  16,  17,  17,
          18,  18,  19,  19,  20,  20,  21,  21,  22,  22,  23,  23,  23,  24,
          24,  25,  25,  26,  26,  26,  27,  27,  28,  28,  29,  29,  29,  30,
          30,  30,  31,  32,  32,  33,  34,  34,  35,  35,  36,  36,  37,  37,
          37,  38,  38,  38,  39,  39,  40,  40,  41,  41,  42,  42,  43,  43,
          44,  44,  45,  46,  46,  47,  47,  48,  48,  48,  49,  49,  50,  50,
          50,  51,  51,  52,  52,  52,  53,  53,  54,  54,  54,  55,  55,  56,
          56,  56,  57,  57,  58,  58,  59,  59,  59,  60,  60,  61,  62,  62,
          62,  63,  63,  63,  63,  64,  65,  66,  67,  67,  68,  68,  69,  69,
          69,  70,  70,  71,  71,  71

DataBatch(x=[166, 79], edge_index=[2, 360], edge_attr=[360, 10], y=[4], batch=[166], ptr=[5])
tensor([[  0,   1,   1,   2,   2,   2,   3,   3,   3,   4,   5,   5,   6,   6,
           6,   7,   7,   8,   8,   9,   9,   9,  10,  10,  11,  11,  11,  12,
          12,  13,  13,  14,  14,  15,  15,  15,  16,  17,  18,  18,  19,  19,
          19,  20,  20,  21,  21,  21,  22,  22,  23,  23,  23,  24,  24,  25,
          25,  26,  26,  26,  27,  27,  28,  28,  29,  29,  30,  30,  31,  31,
          31,  32,  32,  32,  33,  34,  34,  35,  36,  36,  37,  37,  37,  38,
          38,  38,  39,  39,  40,  40,  41,  41,  42,  42,  43,  43,  44,  45,
          45,  45,  46,  46,  47,  47,  47,  48,  48,  49,  49,  49,  50,  50,
          51,  51,  52,  52,  52,  53,  53,  54,  54,  54,  55,  55,  56,  56,
          57,  57,  57,  58,  58,  59,  59,  60,  60,  61,  61,  62,  62,  63,
          63,  63,  64,  64,  65,  65,  65,  66,  66,  66,  67,  68,  68,  69,
          70,  70,  70,  71,  72,  72

DataBatch(x=[168, 79], edge_index=[2, 372], edge_attr=[372, 10], y=[4], batch=[168], ptr=[5])
tensor([[  0,   1,   1,   1,   2,   3,   3,   4,   4,   4,   5,   5,   6,   6,
           7,   7,   7,   8,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,
          12,  12,  13,  13,  14,  14,  15,  15,  15,  16,  16,  17,  17,  18,
          18,  18,  19,  19,  19,  20,  21,  21,  22,  23,  23,  24,  24,  25,
          25,  26,  26,  26,  27,  27,  28,  28,  29,  29,  29,  30,  30,  31,
          31,  31,  32,  33,  33,  34,  34,  34,  35,  35,  36,  36,  36,  37,
          37,  38,  38,  39,  39,  40,  40,  40,  41,  42,  42,  43,  43,  44,
          44,  44,  45,  45,  46,  46,  47,  47,  48,  48,  49,  49,  49,  50,
          50,  50,  51,  52,  52,  52,  53,  53,  54,  55,  55,  55,  56,  56,
          57,  57,  58,  58,  58,  59,  59,  60,  60,  61,  61,  61,  62,  62,
          63,  63,  63,  64,  64,  65,  65,  66,  66,  66,  67,  67,  68,  68,
          69,  69,  70,  70,  71,  71

DataBatch(x=[154, 79], edge_index=[2, 332], edge_attr=[332, 10], y=[4], batch=[154], ptr=[5])
tensor([[  0,   1,   1,   1,   2,   2,   3,   3,   3,   4,   4,   5,   5,   5,
           6,   6,   7,   7,   8,   8,   8,   9,   9,  10,  10,  10,  11,  11,
          12,  12,  12,  13,  13,  14,  14,  14,  15,  16,  16,  17,  17,  17,
          18,  19,  19,  19,  20,  20,  20,  21,  22,  22,  23,  24,  24,  25,
          25,  25,  26,  26,  27,  27,  27,  28,  28,  29,  29,  30,  30,  31,
          31,  32,  32,  32,  33,  34,  34,  35,  35,  36,  36,  37,  37,  37,
          38,  38,  39,  40,  40,  40,  41,  41,  42,  43,  43,  43,  44,  44,
          45,  45,  46,  46,  46,  47,  48,  48,  49,  49,  49,  50,  50,  51,
          51,  51,  52,  52,  53,  53,  53,  54,  54,  55,  55,  56,  56,  56,
          57,  57,  58,  58,  58,  59,  60,  60,  61,  61,  62,  62,  63,  63,
          64,  64,  64,  65,  65,  66,  66,  66,  67,  67,  68,  68,  69,  69,
          69,  70,  71,  71,  72,  72

DataBatch(x=[174, 79], edge_index=[2, 380], edge_attr=[380, 10], y=[4], batch=[174], ptr=[5])
tensor([[  0,   1,   1,   1,   2,   2,   3,   4,   4,   4,   5,   5,   6,   6,
           7,   7,   7,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,
          12,  13,  13,  14,  14,  15,  15,  15,  16,  16,  17,  17,  18,  18,
          18,  19,  19,  19,  20,  21,  22,  22,  23,  23,  24,  24,  25,  25,
          25,  26,  26,  27,  27,  27,  28,  28,  29,  29,  30,  30,  30,  31,
          31,  31,  32,  33,  34,  34,  34,  35,  36,  36,  37,  37,  38,  38,
          39,  39,  40,  41,  41,  41,  42,  42,  43,  43,  44,  44,  44,  45,
          45,  46,  46,  46,  47,  47,  48,  48,  48,  49,  49,  50,  50,  50,
          51,  51,  52,  52,  52,  53,  53,  54,  54,  54,  55,  55,  56,  56,
          57,  57,  58,  58,  58,  59,  59,  59,  60,  61,  61,  62,  62,  63,
          63,  63,  64,  64,  65,  65,  65,  66,  66,  67,  67,  67,  68,  68,
          68,  69,  70,  70,  70,  71

...

DataBatch(x=[163, 79], edge_index=[2, 354], edge_attr=[354, 10], y=[4], batch=[163], ptr=[5])
tensor([[  0,   1,   1,   1,   2,   2,   3,   3,   3,   3,   4,   5,   6,   7,
           7,   7,   8,   8,   9,   9,   9,  10,  10,  11,  11,  11,  12,  12,
          13,  13,  14,  14,  14,  15,  15,  15,  16,  17,  17,  18,  18,  19,
          19,  20,  20,  20,  21,  21,  21,  22,  22,  23,  23,  24,  24,  25,
          25,  26,  26,  27,  27,  27,  28,  28,  29,  29,  29,  30,  30,  31,
          31,  32,  32,  32,  33,  34,  34,  34,  35,  36,  36,  37,  37,  38,
          38,  38,  39,  40,  40,  41,  42,  43,  43,  43,  44,  44,  45,  46,
          46,  46,  47,  47,  48,  48,  48,  49,  50,  50,  51,  51,  52,  52,
          52,  53,  53,  54,  54,  54,  55,  55,  56,  56,  56,  57,  57,  58,
          58,  59,  59,  60,  60,  61,  61,  62,  62,  62,  63,  63,  64,  64,
          65,  65,  66,  66,  66,  67,  68,  68,  69,  69,  70,  70,  70,  71,
          71,  72,  72,  73,  73,  73

DataBatch(x=[160, 79], edge_index=[2, 348], edge_attr=[348, 10], y=[4], batch=[160], ptr=[5])
tensor([[  0,   1,   1,   1,   2,   2,   3,   3,   4,   4,   4,   5,   5,   6,
           6,   6,   7,   7,   8,   8,   8,   9,   9,  10,  10,  11,  11,  11,
          12,  12,  13,  13,  14,  14,  14,  15,  15,  16,  16,  16,  17,  18,
          18,  18,  19,  19,  20,  20,  21,  21,  22,  22,  23,  23,  24,  24,
          25,  25,  26,  26,  26,  27,  27,  28,  28,  28,  29,  29,  30,  30,
          30,  31,  31,  32,  32,  33,  33,  33,  34,  35,  35,  36,  36,  37,
          37,  37,  38,  39,  39,  40,  41,  41,  42,  42,  43,  43,  44,  45,
          45,  46,  46,  47,  47,  47,  48,  49,  49,  49,  50,  50,  51,  51,
          52,  52,  52,  53,  53,  54,  54,  54,  55,  55,  56,  56,  56,  57,
          57,  58,  58,  58,  59,  59,  60,  60,  61,  61,  62,  62,  62,  63,
          63,  63,  64,  65,  65,  66,  67,  67,  68,  68,  69,  69,  69,  70,
          70,  71,  71,  71,  72,  72

DataBatch(x=[158, 79], edge_index=[2, 340], edge_attr=[340, 10], y=[4], batch=[158], ptr=[5])
tensor([[  0,   1,   1,   2,   2,   2,   3,   3,   4,   4,   5,   5,   6,   6,
           7,   7,   7,   8,   8,   9,   9,  10,  10,  11,  11,  12,  12,  13,
          13,  14,  14,  14,  15,  15,  16,  16,  16,  17,  17,  18,  18,  18,
          19,  19,  20,  20,  21,  21,  21,  22,  23,  23,  23,  24,  25,  25,
          26,  26,  27,  27,  27,  28,  28,  29,  29,  29,  30,  30,  31,  31,
          32,  32,  33,  33,  33,  34,  35,  35,  35,  36,  36,  36,  37,  38,
          38,  39,  40,  40,  41,  42,  42,  43,  43,  43,  43,  44,  44,  45,
          46,  46,  47,  47,  48,  48,  48,  49,  49,  50,  50,  50,  51,  51,
          52,  52,  52,  53,  53,  54,  54,  55,  55,  56,  56,  57,  57,  57,
          58,  58,  58,  59,  60,  60,  61,  62,  62,  63,  63,  63,  64,  64,
          65,  65,  65,  66,  66,  66,  67,  68,  68,  69,  70,  70,  70,  71,
          71,  72,  72,  73,  73,  74

DataBatch(x=[166, 79], edge_index=[2, 362], edge_attr=[362, 10], y=[4], batch=[166], ptr=[5])
tensor([[  0,   1,   1,   1,   2,   2,   3,   3,   3,   4,   4,   5,   5,   5,
           6,   6,   7,   7,   8,   8,   9,   9,  10,  10,  10,  11,  12,  12,
          13,  13,  13,  14,  14,  15,  15,  15,  16,  16,  17,  17,  18,  18,
          18,  19,  19,  20,  20,  21,  21,  22,  22,  22,  23,  24,  24,  25,
          25,  26,  26,  26,  27,  27,  28,  28,  29,  29,  29,  30,  30,  31,
          31,  32,  32,  33,  33,  33,  34,  34,  34,  35,  35,  36,  36,  37,
          37,  38,  38,  39,  39,  40,  40,  41,  41,  42,  43,  44,  44,  44,
          45,  45,  46,  46,  46,  47,  47,  48,  48,  48,  49,  49,  50,  50,
          51,  51,  51,  52,  53,  53,  54,  54,  55,  55,  56,  56,  56,  57,
          57,  58,  58,  58,  59,  59,  60,  60,  61,  61,  61,  62,  63,  63,
          64,  64,  65,  65,  65,  66,  66,  67,  67,  68,  68,  69,  69,  70,
          70,  71,  71,  72,  72,  72

DataBatch(x=[155, 79], edge_index=[2, 334], edge_attr=[334, 10], y=[4], batch=[155], ptr=[5])
tensor([[  0,   1,   1,   2,   2,   3,   3,   3,   4,   4,   5,   5,   5,   6,
           6,   7,   7,   7,   8,   8,   9,   9,  10,  10,  10,  11,  11,  11,
          12,  12,  13,  13,  14,  14,  15,  15,  15,  16,  17,  17,  17,  18,
          18,  19,  19,  20,  20,  21,  21,  22,  22,  23,  23,  23,  24,  24,
          25,  25,  26,  26,  26,  26,  27,  27,  28,  28,  28,  28,  29,  30,
          31,  32,  32,  33,  33,  34,  34,  35,  35,  36,  36,  37,  37,  38,
          38,  38,  39,  40,  40,  41,  42,  43,  43,  43,  44,  44,  45,  46,
          46,  46,  47,  47,  48,  48,  49,  49,  49,  50,  50,  51,  51,  52,
          52,  52,  53,  53,  54,  54,  54,  55,  55,  56,  56,  57,  57,  57,
          58,  58,  59,  59,  60,  60,  60,  61,  61,  61,  62,  63,  64,  64,
          65,  65,  66,  66,  67,  67,  67,  68,  68,  69,  69,  69,  70,  70,
          71,  71,  72,  72,  72,  73

In [15]:
gnn_model.eval()

train_target = np.empty((0))
train_y_target = np.empty((0))

for (k, batch) in enumerate(dataloader):

        out = gnn_model(batch)
        train_target = np.concatenate((train_target, out.detach().numpy()[:, 0]))
        train_y_target = np.concatenate((train_y_target, batch.y.detach().numpy()))


tensor([[  0,   1,   1,   2,   2,   3,   3,   4,   4,   5,   5,   6,   6,   7,
           7,   8,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,  12,
          13,  13,  13,  14,  14,  15,  16,  16,  17,  17,  18,  18,  18,  19,
          19,  19,  20,  20,  21,  21,  22,  22,  23,  23,  24,  24,  25,  25,
          26,  26,  26,  27,  27,  27,  28,  29,  29,  29,  30,  30,  31,  31,
          31,  32,  32,  33,  33,  34,  34,  35,  35,  36,  36,  37,  37,  37,
          38,  39,  39,  40,  41,  41,  42,  43,  43,  44,  44,  45,  45,  46,
          46,  47,  47,  48,  48,  49,  49,  49,  50,  50,  51,  51,  51,  52,
          52,  53,  53,  54,  54,  54,  55,  55,  56,  56,  57,  57,  58,  58,
          59,  59,  60,  60,  60,  61,  61,  62,  62,  62,  63,  63,  64,  64,
          65,  65,  65,  66,  67,  67,  68,  68,  68,  69,  69,  69,  70,  71,
          71,  72,  73,  73,  74,  75,  75,  75,  76,  76,  77,  77,  77,  78,
          78,  78,  79,  80,  81,  81,  81,  82,  82

tensor([[  0,   1,   1,   1,   2,   2,   3,   4,   4,   4,   5,   5,   6,   6,
           6,   7,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,  12,
          13,  13,  14,  14,  14,  15,  15,  16,  16,  16,  17,  17,  18,  18,
          19,  19,  20,  20,  21,  21,  22,  22,  22,  23,  23,  24,  24,  24,
          25,  25,  26,  26,  27,  27,  27,  28,  29,  29,  30,  30,  30,  31,
          32,  32,  33,  34,  34,  35,  35,  35,  36,  37,  37,  37,  38,  38,
          39,  39,  40,  40,  41,  41,  41,  42,  42,  43,  43,  43,  44,  44,
          45,  45,  45,  46,  46,  47,  47,  48,  48,  49,  49,  49,  50,  51,
          51,  52,  52,  53,  53,  53,  54,  54,  55,  55,  56,  56,  57,  57,
          58,  58,  59,  59,  60,  60,  60,  61,  61,  62,  62,  62,  63,  63,
          64,  64,  65,  65,  65,  66,  66,  67,  68,  68,  69,  69,  69,  70,
          70,  70,  71,  72,  72,  73,  74,  74,  75,  75,  76,  77,  77,  77,
          78,  78,  79,  79,  80,  80,  80,  81,  81

tensor([[  0,   1,   1,   2,   2,   2,   3,   3,   4,   4,   5,   5,   6,   6,
           7,   7,   7,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,
          12,  13,  13,  14,  14,  15,  15,  15,  16,  16,  17,  17,  17,  18,
          19,  19,  20,  20,  21,  21,  22,  22,  22,  23,  23,  24,  24,  24,
          25,  25,  26,  26,  26,  27,  28,  28,  29,  30,  30,  31,  31,  31,
          32,  32,  33,  33,  34,  34,  34,  35,  35,  36,  37,  37,  38,  38,
          39,  39,  40,  41,  41,  41,  42,  42,  43,  44,  44,  44,  45,  45,
          46,  46,  47,  47,  47,  48,  48,  49,  49,  50,  50,  50,  51,  51,
          52,  52,  52,  53,  53,  54,  54,  55,  55,  56,  56,  57,  57,  57,
          58,  58,  59,  59,  60,  60,  61,  61,  62,  62,  63,  63,  64,  64,
          64,  65,  65,  66,  66,  67,  67,  67,  68,  68,  69,  69,  70,  70,
          70,  71,  71,  72,  72,  72,  73,  73,  74,  74,  75,  75,  76,  76,
          77,  78,  78,  78,  79,  79,  80,  81,  81

tensor([[  0,   1,   1,   2,   2,   2,   3,   3,   4,   4,   5,   5,   5,   6,
           6,   7,   7,   7,   8,   8,   9,   9,  10,  10,  11,  11,  11,  12,
          13,  13,  13,  14,  14,  15,  15,  16,  16,  16,  17,  17,  17,  18,
          18,  19,  19,  20,  20,  21,  21,  22,  22,  23,  23,  24,  24,  25,
          25,  26,  26,  26,  27,  27,  28,  28,  28,  29,  29,  29,  30,  31,
          31,  32,  33,  33,  33,  34,  35,  35,  36,  36,  36,  36,  37,  38,
          39,  40,  40,  41,  41,  42,  42,  42,  43,  43,  44,  44,  45,  45,
          46,  46,  47,  47,  47,  48,  49,  50,  50,  50,  51,  51,  52,  53,
          53,  53,  54,  54,  55,  55,  55,  56,  56,  57,  57,  58,  58,  58,
          59,  60,  60,  61,  61,  61,  62,  62,  63,  63,  63,  64,  64,  64,
          65,  65,  66,  66,  66,  67,  67,  68,  68,  69,  69,  69,  70,  70,
          71,  71,  72,  72,  73,  73,  73,  74,  75,  75,  76,  76,  77,  77,
          77,  78,  78,  79,  79,  80,  80,  80,  80

tensor([[  0,   1,   1,   2,   2,   2,   3,   4,   4,   4,   5,   5,   6,   6,
           7,   7,   8,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,
          12,  13,  13,  14,  14,  15,  15,  15,  15,  16,  16,  16,  17,  17,
          18,  18,  19,  19,  19,  20,  21,  21,  22,  22,  22,  23,  24,  24,
          25,  25,  26,  26,  27,  27,  28,  28,  29,  29,  29,  30,  30,  31,
          31,  31,  32,  32,  33,  33,  33,  34,  34,  34,  35,  36,  37,  37,
          38,  38,  38,  39,  39,  39,  40,  41,  41,  42,  43,  43,  44,  44,
          45,  45,  46,  47,  47,  47,  48,  48,  49,  49,  50,  50,  50,  51,
          51,  52,  52,  53,  53,  53,  54,  54,  55,  55,  55,  56,  56,  57,
          57,  57,  58,  58,  59,  59,  60,  60,  61,  61,  62,  62,  62,  63,
          64,  64,  65,  65,  65,  66,  66,  67,  67,  67,  68,  68,  69,  69,
          69,  70,  71,  71,  72,  73,  73,  73,  74,  74,  75,  75,  76,  76,
          77,  77,  77,  78,  79,  79,  80,  80,  81

...

tensor([[  0,   1,   1,   1,   2,   3,   3,   4,   4,   4,   5,   6,   6,   7,
           7,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,  12,  13,
          13,  14,  14,  15,  15,  15,  16,  17,  17,  18,  18,  18,  19,  19,
          20,  20,  21,  21,  22,  22,  23,  23,  24,  24,  25,  25,  25,  26,
          26,  27,  27,  27,  28,  28,  28,  29,  30,  30,  31,  32,  32,  32,
          33,  33,  34,  34,  35,  35,  36,  36,  37,  37,  38,  38,  39,  40,
          40,  41,  41,  41,  42,  42,  43,  43,  44,  44,  45,  45,  45,  46,
          46,  47,  47,  47,  48,  48,  49,  49,  49,  50,  50,  51,  51,  51,
          52,  52,  53,  53,  53,  54,  54,  54,  55,  55,  56,  56,  57,  57,
          58,  58,  59,  59,  60,  60,  61,  61,  62,  62,  63,  63,  63,  64,
          64,  65,  65,  65,  66,  66,  66,  67,  67,  68,  68,  68,  68,  69,
          70,  71,  72,  72,  73,  73,  74,  74,  75,  75,  75,  76,  76,  76,
          77,  78,  78,  79,  80,  80,  81,  82,  82

tensor([[  0,   1,   1,   1,   1,   2,   3,   4,   4,   5,   5,   5,   6,   7,
           7,   7,   8,   8,   9,   9,  10,  10,  10,  11,  11,  11,  12,  12,
          13,  13,  14,  14,  15,  15,  16,  16,  16,  17,  17,  18,  18,  18,
          19,  19,  20,  20,  20,  21,  21,  22,  22,  23,  23,  24,  24,  24,
          25,  25,  26,  26,  27,  27,  28,  28,  28,  29,  29,  29,  30,  31,
          32,  32,  33,  33,  33,  34,  34,  35,  35,  35,  36,  36,  37,  37,
          37,  38,  39,  39,  40,  41,  41,  41,  42,  42,  43,  43,  44,  44,
          45,  45,  46,  46,  47,  47,  48,  48,  49,  50,  50,  50,  51,  52,
          52,  53,  53,  53,  54,  55,  55,  56,  56,  57,  57,  58,  58,  59,
          59,  59,  60,  60,  61,  61,  61,  62,  62,  63,  63,  64,  64,  64,
          65,  65,  66,  66,  67,  67,  67,  68,  68,  68,  69,  70,  70,  71,
          72,  72,  73,  73,  74,  74,  75,  75,  75,  76,  76,  77,  77,  77,
          78,  78,  79,  79,  80,  80,  80,  81,  81

tensor([[  0,   1,   1,   2,   2,   3,   3,   3,   4,   4,   5,   5,   6,   6,
           6,   7,   7,   8,   8,   9,   9,   9,  10,  10,  11,  11,  11,  12,
          12,  13,  13,  14,  14,  14,  15,  16,  16,  16,  16,  17,  18,  19,
          19,  19,  20,  20,  21,  21,  22,  22,  22,  23,  23,  23,  24,  24,
          25,  25,  26,  26,  27,  27,  28,  28,  29,  29,  30,  30,  31,  31,
          32,  32,  32,  33,  33,  34,  34,  34,  35,  35,  36,  36,  37,  37,
          38,  38,  39,  39,  39,  40,  40,  40,  41,  42,  42,  43,  44,  44,
          45,  45,  46,  46,  47,  48,  48,  48,  49,  49,  50,  50,  51,  51,
          52,  52,  52,  53,  53,  54,  54,  54,  55,  55,  56,  56,  56,  57,
          57,  58,  58,  58,  59,  59,  60,  60,  61,  61,  61,  62,  63,  63,
          64,  64,  64,  65,  66,  66,  67,  67,  67,  68,  68,  69,  69,  69,
          70,  70,  71,  71,  72,  72,  72,  73,  74,  74,  75,  75,  75,  76,
          76,  76,  77,  78,  78,  79,  80,  80,  81

tensor([[  0,   1,   1,   1,   1,   2,   3,   4,   4,   5,   5,   5,   6,   7,
           7,   8,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,  12,
          13,  13,  14,  14,  15,  15,  15,  16,  16,  17,  17,  18,  18,  19,
          19,  20,  20,  21,  21,  21,  22,  22,  23,  23,  24,  24,  24,  25,
          25,  26,  26,  26,  27,  28,  28,  28,  29,  29,  30,  30,  30,  31,
          31,  32,  32,  33,  33,  34,  34,  35,  35,  36,  36,  37,  37,  38,
          38,  38,  39,  40,  40,  41,  42,  43,  43,  43,  44,  44,  45,  45,
          45,  46,  46,  47,  47,  48,  48,  48,  49,  49,  50,  50,  50,  51,
          51,  52,  52,  53,  53,  53,  54,  54,  55,  55,  55,  56,  56,  57,
          57,  58,  58,  59,  59,  60,  60,  60,  61,  61,  62,  62,  62,  63,
          64,  64,  65,  65,  65,  66,  66,  67,  67,  67,  68,  68,  69,  69,
          69,  70,  71,  71,  72,  73,  73,  74,  74,  74,  75,  75,  76,  76,
          77,  77,  77,  78,  79,  79,  80,  80,  81

tensor([[  0,   1,   1,   1,   2,   2,   2,   3,   4,   4,   5,   5,   6,   6,
           7,   7,   7,   8,   8,   9,   9,  10,  10,  10,  11,  11,  12,  12,
          12,  13,  13,  14,  14,  15,  15,  15,  16,  16,  17,  17,  18,  18,
          18,  19,  19,  19,  20,  21,  21,  22,  23,  23,  24,  24,  25,  25,
          26,  26,  26,  27,  27,  28,  28,  28,  29,  29,  30,  30,  31,  31,
          31,  32,  33,  33,  34,  34,  35,  35,  36,  37,  37,  37,  38,  38,
          39,  40,  40,  40,  41,  41,  42,  42,  42,  43,  44,  44,  45,  45,
          46,  46,  46,  47,  47,  48,  48,  48,  49,  49,  50,  50,  50,  51,
          51,  52,  52,  53,  53,  53,  54,  54,  55,  55,  56,  56,  57,  57,
          57,  58,  58,  59,  59,  60,  60,  61,  61,  61,  62,  62,  63,  63,
          63,  64,  64,  65,  65,  65,  66,  66,  67,  67,  68,  68,  68,  69,
          70,  70,  71,  71,  72,  72,  73,  74,  74,  75,  75,  75,  75,  76,
          76,  77,  78,  78,  79,  79,  80,  80,  80

In [16]:
print(train_target)
print(train_y_target)

[0.46228272 0.45370716 0.46793327 0.45939663 0.4670763  0.47585037
 0.47782746 0.49349329 0.47430772 0.49778223 0.48654139 0.48910964
 0.4736948  0.48049    0.48172197 0.4855679  0.47230771 0.44730741
 0.48499364 0.48856544 0.47908139 0.45928273 0.49098176 0.48340109
 0.48482439 0.47048202 0.44693676 0.48298478 0.47922158 0.48301542
 0.50205338 0.47023991 0.45702559 0.45539001 0.46913958 0.47427917
 0.47301799 0.48264062 0.48276412 0.47627053 0.47257006 0.47245836
 0.46220437 0.47771764 0.48362517 0.48952329 0.4826889  0.47544202
 0.47345296 0.47009492 0.47322491 0.47159025 0.47284827 0.47943676
 0.4848645  0.47649822 0.48408732 0.48130643 0.46227869 0.46109989
 0.48120677 0.48342991 0.46552223 0.49067819 0.48993224 0.47147003
 0.46594319 0.47856376 0.51398355 0.48747844 0.4788307  0.47815412
 0.45827818 0.49782327 0.48061016 0.49990454 0.45550492 0.47613418
 0.48211211 0.48088193 0.47256953 0.46414891 0.47110847 0.46256679
 0.48971075 0.47580981 0.46869251 0.48023817 0.48454344 0.4794

In [17]:
# Save the trained model
model_path = 'gnn_model.pth'
torch.save(gnn_model.state_dict(), model_path)